##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [ ]:
%pip install pandas
%pip install matplotlib
%pip install tensorflow
%pip install sklearn

# add commented pip installation lines for packages used as shown above for ease of testing
# the line should be of the format %pip install PACKAGE_NAME

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and
##### have the .csv file inside that. A relative path *must* be used when loading data into pandas

In [ ]:
#connecting to data stored in google drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
filepath = "./data/catA_train.csv"
data = pd.read_csv(filepath)
print(data)
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

FileNotFoundError: [Errno 2] No such file or directory: './data/catA_train.csv'

In [ ]:
filepath = "/content/drive/MyDrive/data"
data = pd.read_csv(filepath, error_bad_lines=False)
/content/drive/MyDrive/data/catA_train.csv
data


### **ALL** Code for machine learning and dataset analysis should be entered below.
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

## Data Cleaning
We cleaned the dataset to remove data with missing values, and store it as a new dataset.

First, we obtained relevant information about the data to decide how we should deal with certain variables.

In [ ]:
# Information about the data
data.info()
data.describe()
data.isna().sum() # this checks how many NA values are there in each column

Then, we start doing the data cleaning. We removed columns where the variable is not relevant to our analysis, and filled NA values with 0 where appropriate.

In [ ]:
# Drop the unnecessary columns
data_cleaned = data.drop(columns = ["SIC Code", "Industry", "8-Digit SIC Description", "Company Description", "Year Found",
                                    "Square Footage", "Fiscal Year End", "LATITUDE", "LONGITUDE",
                                    "Employees (Single Site)", "Parent Company", "Company Status (Active/Inactive)"])

# Cast selected variables to integer
columns_to_fill = ["Employees (Global Ultimate Total)", "Is Domestic Ultimate", "Is Global Ultimate"]

data_cleaned[columns_to_fill] = data_cleaned[columns_to_fill].fillna(0)

data_cleaned.astype({"Employees (Global Ultimate Total)": "int",
                     "Is Domestic Ultimate": "int",
                     "Is Global Ultimate": "int"})

# Replace 'Public Sector' with 'Public' in 'Ownership Type'
data_cleaned['Ownership Type'].replace('Public Sector', 'Public', inplace=True)

# Manipulate Import/Export Status
data_cleaned['Exports'] = 0
data_cleaned['Imports'] = 0
data_cleaned.loc[data_cleaned['Import/Export Status'] == 'Exports', 'Exports'] = 1
data_cleaned.loc[data_cleaned['Import/Export Status'] == 'Imports', 'Imports'] = 1
data_cleaned.loc[data_cleaned['Import/Export Status'] == 'Both Imports & Exports', ['Exports', 'Imports']] = 1
data_cleaned.drop(columns = ['Import/Export Status'], inplace = True)

# fill up the Domestic Ultimate Company column if it is empty
data_cleaned.loc[data_cleaned["Global Ultimate Country"] == "Singapore", "Domestic Ultimate Company"] = data_cleaned["Global Ultimate Company"]

# clean the employees columns
data_cleaned.loc[data_cleaned["Parent Country"] == "Singapore", "Employees (Domestic Ultimate Total)"] = data_cleaned["Employees (Global Ultimate Total)"]

# Fill missing values in 'Employees (Domestic Ultimate Total)' with 0
data_cleaned['Employees (Domestic Ultimate Total)'].fillna(0, inplace=True)

# global ultimate total should not be less than the domestic ultimate total
data_cleaned.loc[data_cleaned["Employees (Global Ultimate Total)"] < data_cleaned["Employees (Domestic Ultimate Total)"], "Employees (Global Ultimate Total)"] = data_cleaned["Employees (Domestic Ultimate Total)"]


NameError: name 'data' is not defined

# XGBoost

In [ ]:
from xgboost import XGBRegressor

We extracted out the relevant categories and used k-fold cross-validation to split the data into training and testing data respectively.

In [ ]:
# Choose the target variable you want to predict
target_variable = 'Sales (Global Ultimate Total USD)'

# Drop non-numeric columns or encode categorical variables as needed
non_numeric_columns = ['AccountID', 'Company', 'Entity Type', 'Parent Country',
                       'Ownership Type', 'Global Ultimate Company',
                       'Global Ultimate Country', 'Domestic Ultimate Company']
data_numeric = data_cleaned.drop(non_numeric_columns, axis=1)

# Extract features (X) and target variable (y)
X = data_numeric.drop([target_variable, '8-Digit SIC Code'], axis=1) # Drop the target variable from features
y = data_numeric[target_variable]

NameError: name 'data_cleaned' is not defined

In [ ]:
xgb_regressor = XGBRegressor(n_estimators = 5000, learning_rate = 1.0, max_depth = 1, objective = "reg:squarederror")

# Store the MSE scores and R^2 Scores for the k-fold CV
mse_scores = []
r2_scores = []

# Split the data into k folds
k = 50
kf = KFold(n_splits = k, shuffle = True, random_state = 42)

for train_index, test_index in kf.split(X):
  # this gets the indices of the training and testing data
  X_train = X.iloc[train_index]
  X_test = X.iloc[test_index]
  y_train= y.iloc[train_index]
  y_test = y.iloc[test_index]

  # fit the data with the model
  xgb_regressor.fit(X_train, y_train)
  y_pred = xgb_regressor.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  r_squared = r2_score(y_test, y_pred)
  mse_scores.append(mse)
  r2_scores.append(r_squared)

# Get the average MSE and R^2 scores
print("MSE score: ", np.mean(mse_scores))
print("R^2 score: ", np.mean(r2_scores))



NameError: name 'X' is not defined

## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list).
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [ ]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform.

All relevant code MUST be included in this function.'''

    result = []


    # Part 1: Data Cleaning
    data_cleaned = hidden_data.drop(columns = ["SIC Code", "Industry", "8-Digit SIC Description", "Company Description", "Year Found",
                                        "Square Footage", "Fiscal Year End", "LATITUDE", "LONGITUDE",
                                        "Employees (Single Site)", "Parent Company", "Company Status (Active/Inactive)"])

    # Cast selected variables to integer
    columns_to_fill = ["Employees (Global Ultimate Total)", "Is Domestic Ultimate", "Is Global Ultimate"]

    data_cleaned[columns_to_fill] = data_cleaned[columns_to_fill].fillna(0)

    data_cleaned.astype({"Employees (Global Ultimate Total)": "int",
                        "Is Domestic Ultimate": "int",
                        "Is Global Ultimate": "int"})

    # Replace 'Public Sector' with 'Public' in 'Ownership Type'
    data_cleaned['Ownership Type'].replace('Public Sector', 'Public', inplace=True)

    # Manipulate Import/Export Status
    data_cleaned['Exports'] = 0
    data_cleaned['Imports'] = 0
    data_cleaned.loc[data_cleaned['Import/Export Status'] == 'Exports', 'Exports'] = 1
    data_cleaned.loc[data_cleaned['Import/Export Status'] == 'Imports', 'Imports'] = 1
    data_cleaned.loc[data_cleaned['Import/Export Status'] == 'Both Imports & Exports', ['Exports', 'Imports']] = 1
    data_cleaned.drop(columns = ['Import/Export Status'], inplace = True)

    # fill up the Domestic Ultimate Company column if it is empty
    data_cleaned.loc[data_cleaned["Global Ultimate Country"] == "Singapore", "Domestic Ultimate Company"] = data_cleaned["Global Ultimate Company"]

    # clean the employees columns
    data_cleaned.loc[data_cleaned["Parent Country"] == "Singapore", "Employees (Domestic Ultimate Total)"] = data_cleaned["Employees (Global Ultimate Total)"]

    # Fill missing values in 'Employees (Domestic Ultimate Total)' with 0
    data_cleaned['Employees (Domestic Ultimate Total)'].fillna(0, inplace=True)

    # global ultimate total should not be less than the domestic ultimate total
    data_cleaned.loc[data_cleaned["Employees (Global Ultimate Total)"] < data_cleaned["Employees (Domestic Ultimate Total)"], "Employees (Global Ultimate Total)"] = data_cleaned["Employees (Domestic Ultimate Total)"]


    # Part 2: Extract features and target variable
    # Choose the target variable you want to predict
    target_variable = 'Sales (Global Ultimate Total USD)'

    # Drop non-numeric columns or encode categorical variables as needed
    non_numeric_columns = ['AccountID', 'Company', 'Entity Type', 'Parent Country',
                          'Ownership Type', 'Global Ultimate Company',
                          'Global Ultimate Country', 'Domestic Ultimate Company']
    data_numeric = data_cleaned.drop(non_numeric_columns, axis=1)

    # Extract features (X) and target variable (y)
    X = data_numeric.drop([target_variable, '8-Digit SIC Code'], axis=1) # Drop the target variable from features
    y = data_numeric[target_variable]


    # Part 3: XGBoost Model
    xgb_regressor = XGBRegressor(n_estimators = 5000, learning_rate = 1.0, max_depth = 1, objective = "reg:squarederror")

    # Store the MSE scores and R^2 Scores for the k-fold CV
    mse_scores = []
    r2_scores = []

    # Split the data into k folds
    k = 50
    kf = KFold(n_splits = k, shuffle = True, random_state = 42)

    for train_index, test_index in kf.split(X):
      # this gets the indices of the training and testing data
      X_train = X.iloc[train_index]
      X_test = X.iloc[test_index]
      y_train= y.iloc[train_index]
      y_test = y.iloc[test_index]

      # fit the data with the model
      xgb_regressor.fit(X_train, y_train)
      y_pred = xgb_regressor.predict(X_test)
      mse = mean_squared_error(y_test, y_pred)
      r_squared = r2_score(y_test, y_pred)
      mse_scores.append(mse)
      r2_scores.append(r_squared)

    # result
    result.append(np.mean(mse_scores))
    result.append(np.mean(r2_scores))

    return result

##### Cell to check testing_hidden_data function

In [ ]:
# This cell should output a list of predictions.
test_df = pd.read_csv(filepath)
test_df = test_df.drop(columns=['Sales (Domestic Ultimate Total USD)'])
print(testing_hidden_data(test_df))

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!